In [ ]:
import os
import cv2
import numpy as np
from glob import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time

In [ ]:
train_images = sorted(glob("../input/cityscapes-500x500/cityscapes500x500/cityscapes500x500/leftImg8bit/train/*/*Img8bit.png"))
train_labels = sorted(glob("../input/cityscapes-500x500/cityscapes500x500/cityscapes500x500/gtFine/train/*/*labelIds.png"))
val_images = sorted(glob("../input/cityscapes-500x500/cityscapes500x500/cityscapes500x500/leftImg8bit/val/*/*Img8bit.png"))
val_labels = sorted(glob("../input/cityscapes-500x500/cityscapes500x500/cityscapes500x500/gtFine/val/*/*labelIds.png"))
test_images = sorted(glob("../input/cityscapes-500x500/cityscapes500x500/cityscapes500x500/leftImg8bit/test/berlin/*"))
print(len(train_images))
print(len(val_images))
print(len(test_images))

In [ ]:
dataset_train = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
dataset_val = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
dataset_train, dataset_val

In [ ]:
def read_png(path, channels=3): 
    # path:  image or label path, if image path, channels=3.  if label path, channels=1
    # because in this training, we read label is  gtFine_labelIds.png(channels=1) ,  not _gtFine_color.png
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=channels)
    return img

def crop_img(img, label):
    concat_img = tf.concat([img, label], axis=-1)
    concat_img = tf.image.resize(concat_img, (280, 280), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    crop_img = tf.image.random_crop(concat_img, [256, 256, 4])
    # tf.image.random_crop， crop a tensor to a given size randomly, here is [256, 256, 4]
    # images and labels are cropped at the same time to maintain consistency,  so need to concat([img, label]) 
    return crop_img[:, :, 0:3], crop_img[:, :, 3:]

def normal(img, label):
    img = tf.cast(img, tf.float32)/127.5 -1
    label = tf.cast(label, tf.int32)
    return img, label

In [ ]:
def load_image_train(img_path, label_path):
    img = read_png(img_path)
    label = read_png(label_path, channels=1)
    if tf.random.uniform(()) > 0.5:
        img = tf.image.flip_left_right(img)
        label = tf.image.flip_left_right(label) 
    img = tf.image.resize(img, (256, 256))
    label = tf.image.resize(label, (256, 256))              
    img, label = normal(img, label)
    return img, label

def load_image_val(img_path, label_path):
    
    img = read_png(img_path)
    label = read_png(label_path, channels=1)
    
    img = tf.image.resize(img, (256, 256))
    label = tf.image.resize(label, (256, 256))
    
    img, label = normal(img, label)
    return img, label

In [ ]:
index = np.random.permutation(len(train_images))
train_images = np.array(train_images)[index]
train_labels = np.array(train_labels)[index]

In [ ]:
BATCH_SIZE = 32
BUFFER_SIZE = 300
train_count = len(train_images)
val_count = len(val_images)
train_step_per_epoch = train_count // BATCH_SIZE
val_step_per_epoch = val_count // BATCH_SIZE
auto = tf.data.experimental.AUTOTUNE

In [ ]:
dataset_train = dataset_train.map(load_image_train, num_parallel_calls=auto)
dataset_val =dataset_val.map(load_image_val, num_parallel_calls=auto)

dataset_train = dataset_train.cache().repeat().shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(auto)
dataset_val = dataset_val.cache().batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
dataset_train, dataset_val

In [ ]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)

def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

In [ ]:
IMAGE_SIZE = 256
NUM_CLASSES = 34
def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    resnet50 = keras.applications.ResNet50(
         weights='imagenet', 
         include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]), 
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)


model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)
# model.summary()
tf.keras.utils.plot_model(model, to_file="my_model.png", show_shapes=True)

In [ ]:
class MeanIoU(tf.keras.metrics.MeanIoU):
#     def __call__(self, y_true, y_pred, sample_weight=None):
#         y_pred = tf.argmax(y_pred, axis=-1)
#         return super().__call__(y_true, y_pred, sample_weight=sample_weight)
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=-1)
        return super().update_state(y_true, y_pred, sample_weight)

In [ ]:
def optimizer_adam_v2():
    # PiecewiseConstantDecay
    step = tf.Variable(0, trainable=False)
    boundaries = [2000, 5000]
    values = [0.0001, 0.00005, 0.000025]

    learning_rate_fn = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)

    # Later, whenever we perform an optimization step, we pass in the step.
    learning_rate_adam = learning_rate_fn(step)

    return keras.optimizers.Adam(learning_rate=learning_rate_adam)

In [ ]:
my_callbacks = [
    # tf.keras.callbacks.EarlyStopping(
        # monitor= 'val_mean_io_u', 
        # min_delta=0, 
        # patience=10, 
        # verbose= 1, 
        # mode='auto', 
        # restore_best_weights=True),
    keras.callbacks.ModelCheckpoint(
        filepath='./{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss',
        save_best_only=False,
        save_weights_only=False)
    # keras.callbacks.LearningRateScheduler(scheduler),
]

loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer=optimizer_adam_v2(),
    loss=loss,
    metrics=['acc', MeanIoU(num_classes=34)]
    # metrics=['accuracy']
)

start = time.time()
# model.load_weights('../input/best-weight/43-0.97.h5')
history = model.fit(dataset_train, 
                   epochs= 20,
                   validation_data=dataset_val,
                   steps_per_epoch=train_step_per_epoch,
                   validation_steps=val_step_per_epoch,
#                    callbacks=[my_callbacks],
                   verbose= 1)
end =time.time()
print(str(int(end - start)))

In [ ]:
#Loading the Colormap
colormap = loadmat(
    "../input/colormap/CS_colormap.mat"
)["colormap"]
colormap = colormap * 100
colormap = colormap.astype(np.uint8)


def infer(model, image_tensor):
    predictions = model.predict(np.expand_dims((image_tensor), axis=0))
    predictions = np.squeeze(predictions)
    predictions = np.argmax(predictions, axis=2)
    return predictions


def decode_segmentation_masks(mask, colormap, n_classes):
    r = np.zeros_like(mask).astype(np.uint8)
    g = np.zeros_like(mask).astype(np.uint8)
    b = np.zeros_like(mask).astype(np.uint8)
    for l in range(n_classes):
        idx = mask == l
        r[idx] = colormap[l, 0]
        g[idx] = colormap[l, 1]
        b[idx] = colormap[l, 2]
    rgb = np.stack([r, g, b], axis=2)
    return rgb


def get_overlay(image, colored_mask):
    image = tf.keras.preprocessing.image.array_to_img(image)
    image = np.array(image).astype(np.uint8)
    overlay = cv2.addWeighted(image, 0.35, colored_mask, 0.65, 0)
    return overlay


def plot_samples_matplotlib(display_list, figsize=(5, 3)):
    _, axes = plt.subplots(nrows=1, ncols=len(display_list), figsize=figsize)
    for i in range(len(display_list)):
        if display_list[i].shape[-1] == 3:
            axes[i].imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        else:
            axes[i].imshow(display_list[i])
    plt.show()


def plot_predictions(images_list, colormap, model):
    for image_file in images_list:
        image_tensor = tf.cast(read_png(image_file), tf.float32)/127.5 -1
        image_tensor = tf.image.resize(images=image_tensor, size=[128, 128])
        prediction_mask = infer(image_tensor=image_tensor, model=model)
        prediction_colormap = decode_segmentation_masks(prediction_mask, colormap, 34)
        overlay = get_overlay(image_tensor, prediction_colormap)
        plot_samples_matplotlib(
            [image_tensor, overlay, prediction_colormap], figsize=(18, 14)
        )


In [ ]:
new_model = tf.keras.models.load_model('my_model.h5')

In [ ]:
test_images = sorted(glob("../input/cityscapes-500x500/cityscapes500x500/cityscapes500x500/leftImg8bit/test/berlin/*"))
plot_predictions(test_images[10:15], colormap, model=model)

In [ ]:
#Loading the Colormap
colormap = loadmat(
    "../input/colormap/CS_colormap.mat"
)["colormap"]
colormap = colormap * 100
colormap = colormap.astype(np.uint8)


def infer(model, image_tensor):
    predictions = model.predict(np.expand_dims((image_tensor), axis=0))
    predictions = np.squeeze(predictions)
    predictions = np.argmax(predictions, axis=2)
    return predictions


def decode_segmentation_masks(mask, colormap, n_classes):
    r = np.zeros_like(mask).astype(np.uint8)
    g = np.zeros_like(mask).astype(np.uint8)
    b = np.zeros_like(mask).astype(np.uint8)
    for l in range(n_classes):
        idx = mask == l
        r[idx] = colormap[l, 0]
        g[idx] = colormap[l, 1]
        b[idx] = colormap[l, 2]
    rgb = np.stack([r, g, b], axis=2)
    return rgb


def get_overlay2(image, colored_mask):
    image = tf.keras.preprocessing.image.array_to_img(image)
    image = np.array(image).astype(np.uint8)
    overlay = cv2.addWeighted(image, 0.1, colored_mask, 0.7, 0)
    return overlay


def plot_samples_matplotlib(display_list, figsize=(5, 3)):
    _, axes = plt.subplots(nrows=1, ncols=len(display_list), figsize=figsize)
    for i in range(len(display_list)):
        if display_list[i].shape[-1] == 3:
            axes[i].imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        else:
            axes[i].imshow(display_list[i])
    plt.show()


def plot_predictions(images_list, colormap, model):
    for image_file in images_list:
        image_tensor = read_image(image_file)
        image_tensor = tf.image.resize(images=image_tensor, size=[128, 128])
        prediction_mask = infer(image_tensor=image_tensor, model=model)
        prediction_colormap = decode_segmentation_masks(prediction_mask, colormap, 34)
        overlay = get_overlay2(image_tensor, prediction_colormap)
        plot_samples_matplotlib(
            [image_tensor, overlay, prediction_colormap], figsize=(18, 14)
        )


In [ ]:
import numpy as np
import cv2 as cv

def rescale_frame(frame_input):
    width = 640
    height = 360
    dim = (width, height)
    return cv2.resize(frame_input, dim, interpolation=cv2.INTER_AREA)

def overlayframe(frame, model, colormap):
    frame = tf.image.resize(images=frame, size=[256, 256])
    frame = tf.cast(frame, tf.float32)/127.5 -1
    prediction_mask = infer(image_tensor=frame, model=model)
    prediction_colormap = decode_segmentation_masks(prediction_mask, colormap, 34)
    overlay = get_overlay2(frame, prediction_colormap)
    overlay = rescale_frame(overlay)
#     overlay = tf.cast(overlay, tf.float32)
    return overlay

cap = cv.VideoCapture('../input/inputavi/input.avi')

fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('output.avi',fourcc, 10.0, (640,360))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    frame = overlayframe(frame, model, colormap)
    out.write(frame)
    # frame = cv.flip(frame, 0)
    # out.write(frame)

cap.release()
out.release()
# cv.destroyAllWindows()